<h1>Sommaire<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Chargement-des-issues-à-analyser" data-toc-modified-id="Chargement-des-issues-à-analyser-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Chargement des issues à analyser</a></span></li><li><span><a href="#Chargement-des-dialogues" data-toc-modified-id="Chargement-des-dialogues-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Chargement des dialogues</a></span></li><li><span><a href="#Analyse-des-insatisfactions" data-toc-modified-id="Analyse-des-insatisfactions-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Analyse des insatisfactions</a></span></li><li><span><a href="#Enregistrement-du-rapport-sur-Github" data-toc-modified-id="Enregistrement-du-rapport-sur-Github-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Enregistrement du rapport sur Github</a></span></li><li><span><a href="#Fermeture-des-issues-analysées" data-toc-modified-id="Fermeture-des-issues-analysées-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Fermeture des issues analysées</a></span></li></ul></div>

In [6]:
%load_ext autoreload
%autoreload 2

from notebook import *

In [58]:
# # Copie les notebooks et supprime les sorties.
# # Décommenter et exécuter cette cellule avant de commiter/pusher
# # les modifications du notebook sur Github.
# copy_and_clean_notebooks()

# Introduction

Dans ce notebook, nous allons analyser les insatisfactions des utilisateurs.

<img src="./data/img/insatisfactions_analysis.png" alt="Analyse des insatisfactions" width="700"/>
<p style="text-align: center; text-decoration: underline;">Analyse des insatisfactions</p>

Une alerte Azure est déclenchée à chaque fois qu'un grand taux d'insatisfactions a été détecté. Cette alerte va elle-même déclencher une Azure function qui va automatiquement créer une Github issue ayant le tag `azure alert`.

<img src="./data/img/insatisfactions_alert.png" alt="Example d’une Github issue générée automatiquement" width="700"/>
<p style="text-align: center; text-decoration: underline;">Example d’une Github issue générée automatiquement</p>

Nous allons commencer par récupérer les informations de ces issues. Nous allons ensuite télécharger les dialogues sources d'insatisfaction. Nous pourrons visualiser et analyser ces dialogue via un widget.

<img src="./data/img/insatisfactions_analysis_widget.png" alt="Widget permettant de visualiser les dialogues et analyser les erreurs" width="700"/>
<p style="text-align: center; text-decoration: underline;">Widget permettant de visualiser les dialogues et analyser les erreurs</p>

Enfin, nous enregistrerons le rapport d'analyse dans une nouvelle Github issue ayant le tag `insatisfactions analysis`.

<img src="./data/img/insatisfactions_analysis_report.png" alt="Example d’un rapport d’analyse sous forme d’une Github issue" width="700"/>
<p style="text-align: center; text-decoration: underline;">Example d’un rapport d’analyse sous forme d’une Github issue</p>

# Chargement des issues à analyser

On commence par récupèrer les issuer de type `azure alert`.

In [4]:
# On charge les variables d'environnement de Github
gh_env = GithubAPIEnv("../P10_04_function/create_github_issue/.env")

In [6]:
# On crée le handler Github
g = Github(gh_env.GITHUB_TOKEN)

# On configure le handler pour le repo du projet
repo = g.get_repo(gh_env.GITHUB_REPO)

In [1]:
# On récupère les issues de type "azure alert"
issues = repo.get_issues(labels=["azure alert"])
print(f"Nombre d'issues à analyser : {issues.totalCount}")

Nombre d'issues à analyser : 1


In [2]:
issue_ids = [f"#{i.number}" for i in issues]
print("Voici les ids des issues que l'on va analyser :", ", ".join(issue_ids))

Voici les ids des issues que l'on va analyser : #37


# Chargement des dialogues

On va utiliser l'API de l'App insights de l'application web pour télécharger les dialogues ayant été annotés comme étant des vrais négatifs donc source d'insatisfactions :

In [46]:
# On charge les variables d'environnement de l'API
# de l'App insights de l'application web.
ai_env = AppInsightsAPIEnv(".env")

In [47]:
data = []
for issue in issues:
    # On charge les données de l'alerte
    issue_body = json.loads(issue.body)
    
    # On charge les dialogues liés à l'alerte
    data.append(get_tn_dialogs(
        ai_env,
        issue_body["start_dt"],
        issue_body["end_dt"]
    ))

In [48]:
# On concatène les données
data = pd.concat(data)
data.shape

(34, 5)

In [49]:
# On supprime les doublons et on met en forme le dataframe
data = data.drop_duplicates(["main_dialog_uuid", "timestamp"])
data = data.sort_values(["main_dialog_uuid", "timestamp"])
data = data.reset_index(drop=True)
data.shape

(34, 5)

Voici un exemple des données précédemment récupérées :

In [50]:
data.head()

,main_dialog_uuid,timestamp,author,session_id,text
0,4b690144-6a3f-11ec-9134-4ae2b1217b29,2021-12-31 13:41:02.230192+00:00,p10-chatbot-bot,I1qjj6+1EFOsJHS9K65QNapgBPr978QJHxKE6iUSeXk=,What else can I do for you?
1,4b690144-6a3f-11ec-9134-4ae2b1217b29,2021-12-31 13:41:06.408448+00:00,You,I1qjj6+1EFOsJHS9K65QNapgBPr978QJHxKE6iUSeXk=,I want to go to Paris from London tomorrow and...
2,4b690144-6a3f-11ec-9134-4ae2b1217b29,2021-12-31 13:41:07.028214+00:00,p10-chatbot-bot,I1qjj6+1EFOsJHS9K65QNapgBPr978QJHxKE6iUSeXk=,Please confirm the following information:\n- Y...
3,4b690144-6a3f-11ec-9134-4ae2b1217b29,2021-12-31 13:41:08.387940+00:00,You,I1qjj6+1EFOsJHS9K65QNapgBPr978QJHxKE6iUSeXk=,Non
4,4f4fdde6-6a3f-11ec-9134-4ae2b1217b29,2021-12-31 13:41:08.776486+00:00,p10-chatbot-bot,I1qjj6+1EFOsJHS9K65QNapgBPr978QJHxKE6iUSeXk=,What else can I do for you?


# Analyse des insatisfactions

Nous allons utiliser un widget afin de visualiser les dialogues et les analyser :

In [4]:
# On crée un nom pour l'analyse
analysis_name = "2022_01_01_analysis"

In [10]:
# On essaie de charger l'analyse si elle existe déjà.
# Cela permet d'interrompre et de reprendre l'analyse.
analyser = InsatisfactionsAnalyser.load(PARQUET_PATH, analysis_name)

# Sinon on en crée une nouvelle
if not analyser:
    analyser = InsatisfactionsAnalyser(data)
    
# On affiche le widget
analyser.display()

In [53]:
# On enregistre l'analyse
analyser.save(PARQUET_PATH, analysis_name)

# Enregistrement du rapport sur Github

Nous allons enregistrer le rapport de l'analyse en créant une Github issue ayant le tag `insatisfactions analysis`.

On pourra alors utiliser ce rapport pour justifier la création de nouvelles user stories, effectuer des corrections de bug ou encore mettre à jour le modèle LUIS avec de nouveaux textes à labelliser.

In [54]:
# On crée le titre et le corps du rapport
report_title = "Insatisfactions analysis"
report_body = analyser.get_report(issue_ids)

In [55]:
try:
    # On récupère le tag spécial pour le rapport d'analyse
    report_label_name = "insatisfactions analysis"
    report_label = repo.get_label(report_label_name)
except:
    # On crée un tag spécial pour le rapport d'analyse
    report_label = repo.create_label(
        report_label_name,
        color="0075ca",
        description="Analysis of insatisfactions"
    )

In [ ]:
# On enregistre le rapport dans une Github issue
report = repo.create_issue(
    title=report_title,
    body=report_body,
    labels=[report_label]
)

# Fermeture des issues analysées

La liste des issues générées par l'alerte des insatisfactions a été enregistrée dans le rapport d'analyse. On peut donc fermer ces issues sur Github.

In [56]:
for issue in issues:
    # On ferme l'issue
    issue.edit(state="closed")